In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
 .appName("RideEventsStreaming") \
 .getOrCreate()

25/06/16 16:19:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/16 16:19:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/16 16:19:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/16 16:19:50 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/16 16:19:50 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/16 16:19:50 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/16 16:19:50 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/06/16 16:19:50 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/06/16 16:19:50 WARN Utils: Serv

In [4]:
from pyspark.sql.types import *

In [5]:
schema = StructType([
 StructField("ride_id", StringType()),
 StructField("tpep_pickup_datetime", StringType()),
 StructField("PULocationID", IntegerType()),
 StructField("driver_id", StringType())
])

In [6]:
rides_df = spark.readStream \
    .schema(schema) \
    .format("json") \
    .option("maxFilesPerTrigger", 1) \
    .load("sparkstream/streaminputs/")

In [7]:
from pyspark.sql.functions import to_timestamp

In [8]:
rides_df = rides_df.withColumn("event_time", to_timestamp("tpep_pickup_datetime"))

In [9]:
from pyspark.sql.functions import window, count

In [10]:
windowed_df = rides_df \
 .withWatermark("event_time", "5 minutes") \
 .groupBy(window("event_time", "10 minutes"), "PULocationID") \
 .agg(count("*").alias("ride_count"))

In [11]:
drivers_df = spark.read.csv("driver_profies.csv", header=True, inferSchema=True)

In [12]:
enriched_df = rides_df.join(drivers_df, on="driver_id", how="left")

In [13]:
low_rated_df = enriched_df.filter("driver_rating < 4.0")

In [14]:
windowed_df.writeStream \
 .outputMode("complete")\
 .format("console") \
 .option("checkpointLocation", "checkpoints/") \
 .start().awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+------------+----------+
|              window|PULocationID|ride_count|
+--------------------+------------+----------+
|{2025-06-10 10:20...|          29|         1|
|{2025-06-10 06:00...|          98|         1|
|{2025-06-10 08:00...|           4|         1|
|{2025-06-10 06:20...|          72|         1|
|{2025-06-10 07:50...|          87|         1|
|{2025-06-10 06:30...|           4|         1|
|{2025-06-10 14:10...|          76|         1|
|{2025-06-10 07:30...|          76|         1|
|{2025-06-10 15:50...|          54|         1|
|{2025-06-10 09:50...|          30|         1|
+--------------------+------------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+------------+----------+
|              window|PULocationID|ride_count|
+--------------------+------------+----------+
|{2025-06-10 10:40...|           6|         1|
|{2025-06-10 08:40...|          28|         1|
|{2025-06-10 10:20...|          29|         1|
|{2025-06-10 07:50...|          12|         1|
|{2025-06-10 06:00...|          98|         1|
|{2025-06-10 08:00...|           4|         1|
|{2025-06-10 06:20...|          72|         1|
|{2025-06-10 07:20...|          71|         1|
|{2025-06-10 12:30...|          74|         1|
|{2025-06-10 07:50...|          87|         1|
|{2025-06-10 06:30...|           4|         1|
|{2025-06-10 15:40...|          16|         1|
|{2025-06-10 07:40...|          46|         1|
|{2025-06-10 14:10...|          76|         1|
|{2025-06-10 07:30...|          76|         1|
|{2025-06-10 11:10...|          11|         1|
|{2025-06-

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+------------+----------+
|              window|PULocationID|ride_count|
+--------------------+------------+----------+
|{2025-06-10 15:40...|          94|         1|
|{2025-06-10 10:40...|           6|         1|
|{2025-06-10 08:40...|          28|         1|
|{2025-06-10 10:20...|          29|         1|
|{2025-06-10 07:50...|          12|         1|
|{2025-06-10 06:00...|          98|         1|
|{2025-06-10 10:50...|          82|         1|
|{2025-06-10 08:00...|           4|         1|
|{2025-06-10 14:10...|          82|         1|
|{2025-06-10 06:20...|          72|         1|
|{2025-06-10 07:50...|          49|         1|
|{2025-06-10 07:20...|          71|         1|
|{2025-06-10 12:30...|          74|         1|
|{2025-06-10 07:50...|          87|         1|
|{2025-06-10 06:30...|           4|         1|
|{2025-06-10 15:40...|          16|         1|
|{2025-06-

KeyboardInterrupt: 

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+------------+----------+
|              window|PULocationID|ride_count|
+--------------------+------------+----------+
|{2025-06-10 07:10...|          28|         1|
|{2025-06-10 15:40...|          94|         1|
|{2025-06-10 10:40...|           6|         1|
|{2025-06-10 08:40...|          28|         1|
|{2025-06-10 10:20...|          29|         1|
|{2025-06-10 07:50...|          12|         1|
|{2025-06-10 12:30...|          29|         1|
|{2025-06-10 06:00...|          98|         1|
|{2025-06-10 13:10...|          83|         1|
|{2025-06-10 10:50...|          82|         1|
|{2025-06-10 08:00...|           4|         1|
|{2025-06-10 14:10...|          82|         1|
|{2025-06-10 15:10...|          64|         1|
|{2025-06-10 08:30...|          34|         1|
|{2025-06-10 06:20...|          72|         1|
|{2025-06-10 07:50...|          49|         1|
|{2025-06-

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+------------+----------+
|              window|PULocationID|ride_count|
+--------------------+------------+----------+
|{2025-06-10 07:10...|          28|         1|
|{2025-06-10 15:40...|          94|         1|
|{2025-06-10 15:00...|          98|         1|
|{2025-06-10 06:50...|          15|         1|
|{2025-06-10 10:40...|           6|         1|
|{2025-06-10 07:00...|          50|         1|
|{2025-06-10 14:30...|          68|         1|
|{2025-06-10 08:40...|          28|         1|
|{2025-06-10 10:20...|          29|         1|
|{2025-06-10 07:50...|          12|         1|
|{2025-06-10 12:30...|          29|         1|
|{2025-06-10 06:00...|          98|         1|
|{2025-06-10 13:10...|          83|         1|
|{2025-06-10 10:50...|          82|         1|
|{2025-06-10 08:00...|           4|         1|
|{2025-06-10 14:10...|          82|         1|
|{2025-06-

In [14]:
from pyspark.sql.functions import desc

In [15]:
windowed_df.orderBy(desc("ride_count")).limit(5)

DataFrame[window: struct<start:timestamp,end:timestamp>, PULocationID: int, ride_count: bigint]

In [16]:
filtered_df = enriched_df.filter("driver_rating < 4.0")

In [17]:
from pyspark.sql.functions import current_timestamp

In [17]:
enriched_df.withColumn("processing_time", current_timestamp()) \
 .select("event_time", "processing_time", "driver_id") \
 .writeStream.format("console") \
 .start()

25/06/16 16:19:03 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4484ff58-9001-4ff9-960b-86b370cee274. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+--------------------+---------+
|         event_time|     processing_time|driver_id|
+-------------------+--------------------+---------+
|2025-06-10 08:01:36|2025-06-16 16:19:...|     D009|
|2025-06-10 10:27:29|2025-06-16 16:19:...|     D005|
|2025-06-10 07:51:57|2025-06-16 16:19:...|     D018|
|2025-06-10 07:34:57|2025-06-16 16:19:...|     D014|
|2025-06-10 06:34:42|2025-06-16 16:19:...|     D003|
|2025-06-10 09:58:48|2025-06-16 16:19:...|     D017|
|2025-06-10 06:28:59|2025-06-16 16:19:...|     D007|
|2025-06-10 15:55:13|2025-06-16 16:19:...|     D008|
|2025-06-10 14:10:39|2025-06-16 16:19:...|     D009|
|2025-06-10 06:07:05|2025-06-16 16:19:...|     D006|
+-------------------+--------------------+---------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+--------------------+---------+
|         

25/06/16 16:19:19 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /172.31.64.14:56132 is closed
25/06/16 16:19:19 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to get executor loss reason for executor id 1 at RPC address 172.31.64.14:56154, but got no response. Marking as agent lost.
java.io.IOException: Connection from /172.31.64.14:56132 closed
	at org.apache.spark.network.client.TransportResponseHandler.channelInactive(TransportResponseHandler.java:146)
	at org.apache.spark.network.server.TransportChannelHandler.channelInactive(TransportChannelHandler.java:117)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:262)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:248)
	at io.netty.channel.AbstractChannelHandlerContext.fireChannelInactive(AbstractChannelHandlerContext.java:241)
	at io.netty.channel.ChannelInboundHandl

In [18]:
running_counts = enriched_df.groupBy("driver_id").count()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
FileSource[sparkstream/streaminputs/]